In [ ]:
import cv2
import numpy as np
import tensorflow as tf

# Cargar modelo correcto
model = tf.keras.models.load_model(
    r"C:\UDEM\Semestre5\IA\CNNproyecto\final_cnn_model.h5"
)
print("Modelo cargado correctamente.")

# Función para preprocesar cada dígito
def preprocess_digit(roi):
    roi = cv2.resize(roi, (28, 28))
    roi = roi.astype("float32") / 255.0
    roi = 1.0 - roi                      
    roi = np.expand_dims(roi, axis=-1)   
    roi = np.expand_dims(roi, axis=0)    
    return roi

# Abrir cámara
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("No se pudo abrir la cámara.")
    exit()

kernel = np.ones((3, 3), np.uint8)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (7, 7), 0)
    _, thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    dilated = cv2.dilate(thresh, kernel, iterations=2)

    contours, _ = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)

        if w < 20 or h < 20:
            continue

        roi = thresh[y:y+h, x:x+w]

        digit = preprocess_digit(roi)

        preds = model.predict(digit, verbose=0)
        pred_class = np.argmax(preds)
        pred_prob = float(np.max(preds))

        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(
            frame,
            f"{pred_class} ({pred_prob*100:.1f}%)",
            (x, y - 10),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.9,
            (0, 255, 0),
            2
        )

    cv2.imshow("Predicción en vivo – CNN", frame)
    cv2.imshow("Umbral", thresh)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



Modelo cargado correctamente.
